# Lab Inferential statistics - ANOVA

## Library loading

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as stats

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Part 1

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data anova_lab_data.xlsx file in the files_for_lab folder

State the null hypothesis
State the alternate hypothesis
What is the significance level
What are the degrees of freedom of model, error terms, and total DoF


Is there any difference in the mean etching rate for different level of power. (reject if there is difference, accept if there is no difference)

Null hypothesis of the Anova: the mean etching rate for different level of power are the same


Alternate hypothesis: the mean etching rate for different level of power are not the same

Significance level: 0.05

It's not double/two-sided

Degrees of freedom: 3 groups - 1 = 2 groups

Set up degrees of freedom with demoninators
Total observations: 15- nr of groups (3) = 12 demoninators
==> Check the F-table (df1 =2 /df2 = 12) : critical value = 3.8853

## Part 2

In this section, use the Python to conduct ANOVA.

What conclusions can you draw from the experiment and why?

In [4]:
# Import libraries
from sklearn.feature_selection import VarianceThreshold
import pandas as pd 
from sklearn.feature_selection import RFE
from sklearn import linear_model
import math

In [39]:
df = pd.read_excel('/content/drive/MyDrive/unit8/day3/lab-inferential-statistics-anova/files_for_lab/anova_lab_data.xlsx')
df.head(20)

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [40]:
df.columns

Index(['Power ', 'Etching Rate'], dtype='object')

In [41]:
df.columns = ['Power','Etching Rate']

In [42]:
df.columns

Index(['Power', 'Etching Rate'], dtype='object')

In [31]:
!pip install researchpy

In [32]:
import researchpy as rp

In [43]:
rp.summary_cont(df['Etching Rate'].groupby(df['Power']))

,N,Mean,SD,SE,95% Conf.,Interval
Power,,,,,,
160 W,5,5.792,0.3199,0.1431,5.3948,6.1892
180 W,5,6.238,0.4343,0.1942,5.6987,6.7773
200 W,5,8.318,0.6696,0.2995,7.4866,9.1494


#### Computing the global mean and the means of each group.

In [44]:
gm = np.mean(df['Etching Rate'])
print("Global mean",gm)

Global mean 6.782666666666667


In [45]:
# Mean of every group:
group_averages = [(group,np.mean(df[df['Power'] == group]['Etching Rate'])) for group in list(df['Power'].unique())] 
print("The group means are: ",group_averages)

The group means are:  [('160 W', 5.792), ('180 W', 6.2379999999999995), ('200 W', 8.318)]


#### Compute the numerator of F:

In [46]:
numerator = 0
for group in group_averages:
    numerator += len(df[df['Power'] == group[0]]['Etching Rate'].values) * ( group[1] - gm )**2
numerator = numerator / ( len(df['Power'].unique()) - 1 ) 

print("\hat{S}_{t}^{2} is: ", numerator)

\hat{S}_{t}^{2} is:  9.088326666666667


#### Compute the denominator:

In [47]:
denominator = 0
for group in group_averages: 
    for value in df[df['Power'] == group[0]]['Etching Rate'].values:
        denominator += (value - group[1])**2
denominator = denominator / ( len(df) - len(df['Power'].unique()) )

print("\hat{S}_{e}^{2} is: ",denominator)

\hat{S}_{e}^{2} is:  0.24643666666666644


#### Compute the ratio:

In [48]:
F = numerator / denominator

print("The F statistic is: ",F)

The F statistic is:  36.87895470100503


Assuming a confidence interval of 95%, our significance level  α  = 0.05. The degrees of freedom in the numerator is: (number of groups - 1) = 3 - 1 = 2. The number of degrees of freedom for the denominator is: ( number of records - number of groups ) = 15 - 3 = 12.

If we go to the F-Table, we get that the critical value for F is: 3.8853

#### Using the critical value

Now taking the decission using the F-critical value:

As our F-statistic (36.88) > F-ctitical value (3.88), we reject the H0

In [49]:
import scipy.stats as stats

stats.f_oneway(df['Etching Rate'][df['Power'] == '160 W'],
               df['Etching Rate'][df['Power'] == '180 W'],
               df['Etching Rate'][df['Power'] == '200 W'])


F_onewayResult(statistic=36.87895470100505, pvalue=7.506584272358903e-06)

P-value < significance level ==> we reject the H0